# Procesamiento de datos
Fecha: 4 Diciembre 2024

## Indice
1. Comentario sobre append de listas <br>
2. Introducción a la estadística <br>
2.1 Medidas de tendencia central <br>
2.2 Cuartiles y percentiles <br>
2.3 Medidas de dispersión <br>
2.4 Correlación <br>
3. **Groupby** <br>
4. **Limpieza del set de datos** <br>
4.1 **Valores faltantes** <br>
4.2 **Registros duplicados** <br>
4.3 **Columnas correlacionadas** <br>


Antes de empezar, importamos los paquetes que vamos a utilizar:

In [ ]:
import pandas as pd
import numpy as np

# Group by

Por Groupby nos estamos refiriendo a un proceso que incluye uno o mas de los siguientes pasos:
1. **Dividir** (split) los datos en grupos en base a algún criterio.
2. **Aplicar** (apply) una función a un grupo de forma independiente.
3. **Combinar** (combine) los resultados en una estructura.

En el segundo paso, se pueden realizar distintas operaciones:

- **Agregación**: Aplicar funciones estadisticas a cada grupo, como sumar todos los elementos, calcular la media, su tamano o ver el número de elementos (.sum(), .mean(), .size, .count()).

- Transformación: Realizar cálculos específicos para grupos que devuelven un objeto indexado. Por ejemplo: estandarizar datos dentro de un grupo (z score), o rellenar valores faltantes con valores derivados de cada grupo.

- Filtrado: Descartar algunos grupos, de acuerdo a calculos en cada grupo evaluados como True o False. Por ejemplo: Descartar datos que pertenecen a grupos con pocos miembros o filtrar datos basado en sum o mean.

Groupby es una herramienta muy potente para el análisis de datos. Si quieres saber mas, visita la documentación de pandas: https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

Para ver como funciona, vamos a crear un dataframe de prueba:

In [ ]:
# dataframe 1
df1 = pd.DataFrame([
    ['1','Estudio', 2, 39],
    ['2','Apartamento', 2, 65],
    ['3','Estudio', 2, 41],
    ['4','Casa de campo', 3, 120],
    ['5','Estudio', 1, 35],
    ['5','Apartamento', 2, 70],
    ['7','Apartamento', 3, 83],
    ['8','Apartamento', 4, 90],
    ['9','Casa de campo', 5, 122], 
    ['10','Estudio', 1, 25],
    ['11','Estudio', 1, 27],
    ['12', 'Nave industrial', 1, 150],
    ['13','Casa de campo', 4, 95], 
    ['14', 'Nave industrial', 1, 130],
    ['15','Estudio', 1, 25]],
    columns=['id', 'Inmueble', 'Habitaciones', 'metros cuadrados'])


df1.head()

In [ ]:
# Ejemplo:
df1.groupby(["Inmueble"])

Si usamos groupby, el objeto que tenemos es un objeto groupby de dataframe

In [ ]:
type(df1.groupby(["Inmueble"]))

Aunque los grupos ya estarían creados, tenemos que elegir una operación para visualizarlos. Por ejemplo, podemos hacer la suma de habitaciones y metros cuadrados que hay por cada inmueble:

In [ ]:
df1.groupby(["Inmueble"]).sum()

Es decir, sabemos que hay un total de 11 habitaciones pertenecientes a apartamentos, y que en total, todos los apartamentos juntan 308 metros cuadrados.

Si os fijais, los resultados se han devuelto en orden alfabético. Si queremos que salgan en orden de aparición, podemos escribir sort=False.

In [ ]:
df1.groupby(["Inmueble"], sort=False).sum()

Quizá solo nos interesa la información sobre los metos cuadrados que hay asociados a cada tipo de inmueble:

In [ ]:
df1.groupby(["Inmueble"])[['metros cuadrados']].sum() # notad aquí que metros cuadrados tiene dos brakets [[...]]

También podemos obtener, cuantas viviendas hay por cada tipo de inmueble:

In [ ]:
df1.groupby(["Inmueble"]).count()

Puedo obtener esta tabla también en función del número de habitaciones además del tipo de inmueble:

In [ ]:
df1.groupby(["Habitaciones", 'Inmueble']).count()

Como hemos visto arriba, podemos además mostras solo los metros cuadrados (y no el id). Esta tabla puede hacerse mas compacta con ayuda de unstack():

In [ ]:
df1.groupby(["Habitaciones", 'Inmueble'])[['metros cuadrados']].count().unstack()

Aquí también puedo pasar funciones estadísticas, por ejemplo, puedo obtener la media de metros cuadrados según el tipo de inmueble y su número de habitaciones:

In [ ]:
df1.groupby(["Habitaciones", 'Inmueble'])[['metros cuadrados']].mean().unstack()

**Ejercicio** : Si en vez de .count() o mean(), usamos .sum(), .size(), .std(), .nunique(), .describe(), que obtenemos?

# Limpieda del set de datos

## 2.1 Valores faltantes


En ocasiones, trabajando con sets de datos reales podemos encontrar que faltan valores, es decir, **valores que no están definidos o que tienen un valor sin sentido**. Esto puede ser debido a cualquier acontecimiento, como por ejemplo errores en la transcripción de los datos o a la falta de predisposición a responder a ciertas preguntas en una encuesta. 
Los valores faltantes **pueden ser por tanto aleatorios o no aleatorios**. Los primeros, disminuyendo el tamano de las muestras, pueden perturbar el análisis de datos. Los segundos, además, producen una disminución de la representatividad de la muestra.

Una forma habitual en la que encontraremos esto es mediante la presencia de valores NaN (Not-a-Number). Por ejemplo:


In [ ]:
# dataframe 1
df1 = pd.DataFrame([
    ['1','Estudio', '?', 39],
    ['2','Apartamento', np.nan, 65],
    ['3','Estudio', np.nan, 41],
    ['4',np.nan, 3, 120],
    ['5','Estudio', 1, np.nan],
    ['-','Apartamento', 2, 70],
    ['7','Apartamento', 3, np.nan],
    ['8','Apartamento', 4, 90],
    ['9','Casa de campo', 5, 122], 
    ['10','Estudio', 1, 25],
    ['11','Estudio', 1, 27],
    ['12', 'Nave industrial', 1, 150],
    ['13','Casa de campo', 4, 95], 
    ['14', 'Nave industrial', 1, 130],
    ['15','Estudio', 1, np.nan]],
    columns=['id', 'Inmueble', 'Habitaciones', 'metros cuadrados'])


df1.head(7)

Como vimos anteriormente, hay varias opciones para averiguar si tenemos valores faltantes como NaN en nuestro data set. Una sería usar .info(); nos devuelve una columna con los valores no nulos que podemos comparar con el tamano total del dataset

In [ ]:
print('Número de filas: ' + str(df1.shape[0]))

df1.info()

O podemos utilizar la función isnull(). ( Nota: Ojo que isnull() busca NaN, no ceros).

In [ ]:
print(df1.isnull().sum())

Sin embargo, puede ocurrir que los valores faltantes vengan especificados por caracteres diferentes, como '?', '%', etc. o valores que no tengan sentido, como por ej '0', '-1' o '9999' si estamos mirando una columna con la altura de personas.

Si os fijais, en el dataset anterior df1, hay un '?' que no ha sido detectado como NaN. 

In [ ]:
df1.head(3)

Para poder trabajar con ese valor faltante de forma mas explicita, podemos reemplazarlo por NaN de la siguiente manera:

In [ ]:
df1.replace('?', np.nan, inplace = True)
df1.head(3)

Como podemos trabajar entonces con valores faltantes? Hay varias opciones:
1. Descarte: <br>
    1.1 Eliminar las columnas que contienen valores faltantes <br>
    1.2 Eliminar las filas que contienen valores faltantes <br>
2. Imputación <br>
    2.1 Rellenar los valores faltantes <br>

3. Dejarlos tal cual y asegurarnos de que cuando hagamos calculos con las filas o columnas que los contengan, no serán tomados en cuenta.

### 1) Descarte de valores faltantes 
1.1) Podemos eliminar que continen valores faltantes, pero este es un caso muy extremo  y solo podría convenir con las columnas que tienen muchos valores faltantes.

In [ ]:
updated_df = df1.dropna(axis=1)
updated_df.head()

En nuestro caso de hecho, perdemos dos columnas enteras.

1.2) Si tenemos muchas filas, y comprobamos que no va a afectar a la representatividad de la muestra, podemos eliminar las filas que contienen NaN:

In [ ]:
updated_df = df1.dropna(axis=0)
updated_df.head()

### 2) Imputación de valores faltantes

Una razón para imputar datos faltantes es que hay modelos de aprendizaje automático que probablemente quieras usar que te devolverán un error si les pasas valores NaN.

La forma mas sencilla de solucionar esto con imputación, sería darles un valor concreto, como por ejemplo 0. Sin embargo, esto puede reducir el accuracy de tu modelo de forma significativa. Hay por tanto varias opciones para imputar valores:

Si el valor faltante corresponde a una **variable numérica**:
- Rellenar los valores faltantes con 0 o -9, o cualquier valor que no fuese a aparecer en el set de datos. Esto puede hacerse de manera que la máquina reconozca que el dato no es real.
- Rellenar los valores faltantes con la media o mediana de los demás valores de la fila / columna.

Si el valor faltante corresponde a una **variable categórica**:
- Rellenar los valores faltantes con la moda.
- Rellenar valores faltantes con un nuevo tipo.

Una herramienta muy útila para esto es **.fillna()**. Veamos algunos ejemplos:

**Ejemplo: ** Reemplazar los valores faltantes de la columna Habitaciones por un número, por ej, el cero.

In [ ]:
# Hacemos una copia de nuestro dataframe con la que jugar
updated_df = df1.copy()

# Podemos rellenar nuestro dataframe de varias formas, por ej.

#Utilizando inplace = True:
updated_df['Habitaciones'].fillna(0, inplace = True) # Si queremos reemplazar valores en la columna Habitaciones

# Reemplazando la columna:
updated_df['Habitaciones'] = updated_df['Habitaciones'].fillna(0)

# Si queremos reemplazar valores en todo el dataframe, no especificamos columna:
#updated_df.fillna(0, inplace = True) 

# Comprobamos los valores faltantes que quedan ahora:
print(updated_df.info())

updated_df

**Ejemplo: ** Reemplazar los valores faltantes en la columna Habitaciones por la media o mediana de los valores en la misma columna.

In [ ]:
# Hacemos una copia de nuestro dataframe con la que jugar
updated_df = df1.copy()

# Primero necesitamos que todos nuestros elementos faltantes esten como NaN:
updated_df.replace('?', np.nan, inplace = True)

updated_df['Habitaciones'] = updated_df['Habitaciones'].fillna(updated_df['Habitaciones'].mean())
#updated_df['Habitaciones'].fillna(updated_df['Habitaciones'].mean(), inplace = True)


updated_df

**Ejemplo: ** Reemplazar los falores faltantes en la columna Inmueble por la moda de los valores en dicha columna.

In [ ]:
# Hacemos una copia de nuestro dataframe con la que jugar
updated_df = df1.copy()

updated_df['Inmueble'].fillna(updated_df['Inmueble'].mode()[0], inplace=True)
# updated_df['Inmueble'] = updated_df['Inmueble'].fillna(updated_df['Inmueble'].mode()[0]) # Alternativamente

updated_df

**Ejemplo: ** Reemplazar los falores faltantes en la columna Inmueble por un nuevo string.

In [ ]:
# Hacemos una copia de nuestro dataframe con la que jugar
updated_df = df1.copy()

updated_df['Inmueble'].replace(np.nan, 'falta la info', inplace = True)
updated_df

**Ejercicio** Reemplaza los valores faltantes en la columna metros cuadrados por (a) un valor facil de diferenciar, (b) la media, (c) la mediana.

**Ejercicio** Reemplaza el valores faltantes en la columna id por el valor que tu estimes adecuado.

Otra posibilidad para rellenar valores faltantes, que no veremos hoy, es hacer uso de modelos de regresión.

## 2.2 Registros duplicados

Otro problema que podemos encontrar en datasets reales es la presencia de valores duplicados.

Los valores duplicados no van a hacer que el modelo de Machine Learning no pueda ejecutarse, como ocurriría con los valores faltantes para muchos tipos de modelos. En cambio, **los valores duplicados pueden trastocar la interpretación de los resultados**, por ejemplo:

- Dándole más peso, artificialmente, a los registros duplicados, haciendo que el modelo se centre más en ellos sin motivo.

- En el caso en el que un valor duplicado aparezca tanto en el conjunto de entrenamiento como en el conjunto de test, estaremos frente a un caso de data leaking . **Data leaking** es el fenómeno por el que parte de la información del conjunto de test se cuela en el entrenamiento. Este fenómeno puede comprometer críticamente nuestro proyecto de Machine Learning.

Vamos a cargar un dataframe de ejemplo para ver como podemos jugar con registros duplicados:



In [ ]:
# dataframe 1
df1 = pd.DataFrame([
    ['1','Estudio', 2, 39],
    ['2','Apartamento', 2, 65],
    ['3','Estudio', 2, 41],
    ['3','Estudio', 2, 41],
    ['3','Estudio', 2, 41],
    ['4','Casa de campo', 3, 120],
    ['5','Estudio', 1, 35],
    ['5','Apartamento', 2, 70],
    ['7','Apartamento', 3, 83],
    ['8','Apartamento', 4, 90],
    ['9','Casa de campo', 5, 122], 
    ['10','Estudio', 1, 25],
    ['10','Estudio', 1, 25],
    ['10','Estudio', 1, 25],
    ['10','Estudio', 1, 25],
    ['11','Estudio', 1, 27],
    ['12', 'Nave industrial', 1, 150],
    ['13','Casa de campo', 4, 95], 
    ['14', 'Nave industrial', 1, 130],
    ['15','Estudio', 1, 25]],
    columns=['id', 'Inmueble', 'Habitaciones', 'metros cuadrados'])


df1.head(7)

Lo primero sería ver si un registro esta duplicado o no. Para ello podemos usar .duplicated()

In [ ]:
df1.duplicated()

El número de registros duplicados se puede obtener sumando los True despues de usar duplicated().

In [ ]:
df1.duplicated().sum()

En ocasiones, puede ser interesante estudiar que registros estan duplicados (como por ejemplo para entender que errores se han producido en la adquisición de datos para que se diese este caso).
Podemos utilizar el resultado del método duplicated () como una máscara para filtrar sólo los valores duplicados.


In [ ]:
df1_duplicados = df1[df1.duplicated()]
df1_duplicados

Para elimiar los elementos duplicados de nuestro DataFrame podemos utilizar el método drop_duplicates().

In [ ]:
df1.drop_duplicates(inplace = True)

print('Número de valores duplicados: ' + str(df1.duplicated().sum()))

df1

## 2.3 Columnas correlacionadas


En algunas ocasiones, encontraremos columnas muy correlacionadas entre sí, o algunas que serán directamente iguales.

Si hay dos columnas iguales, podemos pensar en eliminarlas. Puede que se haya producido un error en la
adquisición de datos duplicando algún valor.

Es posible que encontremos columnas que no sean exactamente iguales, pero que estén totalmente correlacionadas. Por ejemplo: si tenemos una columna de peso en libras y otra de peso en kilogramos.

En el caso de columnas que estén muy correlacionadas, debemos pensar si esta correlación es intrínseca o si es
accidental. Que algo haya estado correlacionado en el pasado no significa que lo vaya a estar en el futuro, ni que exista una relación causal entre ambas variables.
En cualquier caso, podemos probar a eliminar una columna si está muy correlacionada con otra y ver si eso mejora la
predicción de nuestro modelo. 

Veamos un ejemplo sencillo:

In [ ]:
# dataframe 1
df1 = pd.DataFrame([
    ['1','Estudio','Estudio', 2, 39, 390000],
    ['2','Apartamento','Apartamento', 2, 65, 650000],
    ['3','Estudio','Estudio', 2, 41, 410000],
    ['4','Casa de campo','Casa de campo', 3, 120, 1200000],
    ['5','Estudio','Estudio', 1, 35, 350000]],
    columns=['id', 'Inmueble', 'tipo de inmueble', 'Habitaciones', 'metros cuadrados', 'centimetros cuadrados'])


df1

** Ejemplo:** Eliminar una columna duplicada.

In [ ]:
df1.drop('tipo de inmueble',axis = 1,  inplace = True)
df1

Podemos usar el método .corr() para calcular la correlación entre todas las columnas del DataFrame:

In [ ]:
df1.corr()

También para calcular la correlación entre dos columnas:

In [ ]:
df1['metros cuadrados'].corr(df1['centimetros cuadrados'])